# [TR-000] Complex square roots

<!-- cspell:disable -->

This notebook investigates how to write a square root function in {mod}`sympy` that computes the positive square root for negative values. The lambdified version of this 'complex square root' should have the same behavior for each computational backend.

In [ ]:
%%sh
pip install black==21.5b2 jax==0.2.13 jaxlib==0.1.67 numpy==1.19.5 sympy==1.8 > /dev/null

In [ ]:
import inspect

import jax
import jax.numpy as jnp
import numpy as np
import sympy as sp
from black import FileMode, format_str

## Negative input values

When using {mod}`numpy` as back-end, {mod}`sympy` lambdifies a {func}`~sympy.functions.elementary.miscellaneous.sqrt` to a {doc}`numpy.sqrt() <reference/generated/numpy.sqrt>`:

In [ ]:
x = sp.Symbol("x")
sqrt_expr = sp.sqrt(x)
sqrt_expr

In [ ]:
np_sqrt = sp.lambdify(x, sqrt_expr, "numpy")
source = inspect.getsource(np_sqrt)
print(source)

As expected, if input values for the {doc}`numpy.sqrt() <reference/generated/numpy.sqrt>` are negative, {mod}`numpy` raises a {class}`RuntimeWarning` and returns `NaN`:

In [ ]:
sample = np.linspace(-1, 1, 5)
np_sqrt(sample)

If we want {mod}`numpy` to return imaginary numbers for negative input values, one can use {class}`complex` input data instead (e.g. {doc}`numpy.complex64 <numpy:reference/arrays.scalars>`). Negative values are then treated as lying just above the real axis, so that their square root is a positive imaginary number:

In [ ]:
complex_sample = sample.astype(np.complex64)
np_sqrt(complex_sample)

A {func}`sympy.sqrt <sympy.functions.elementary.miscellaneous.sqrt>` lambdified to [JAX](https://jax.rtfd.io) exhibits the same behavior:

In [ ]:
jax_sqrt = jax.jit(sp.lambdify(x, sqrt_expr, jnp))
source = inspect.getsource(jax_sqrt)
print(source)

In [ ]:
jax_sqrt(sample)

In [ ]:
jax_sqrt(complex_sample)

**There is a problem with this approach though**: once input data is complex, _all_ square roots in a larger expression (some amplitude model) compute imaginary solutions for negative values, while this is not always the desired behavior.

Take for instance the two square roots appearing in {func}`~ampform.dynamics.phase_space_factor` --- does the $\sqrt{s}$ also have to be evaluatable for negative $s$?

## Complex square root

Numpy also offers a special function that evaluates negative values even if the input values are real: {func}`numpy.lib.scimath.sqrt`:

In [ ]:
from numpy.lib.scimath import sqrt as csqrt

csqrt(-1)

However, the {mod}`jax.numpy` API does not interface to {mod}`numpy.lib.scimath`, nor can {func}`numpy.lib.scimath.sqrt` be decorated with {func}`jax.jit`:

In [ ]:
jax_csqrt = jax.jit(csqrt, backend="cpu")
jax_csqrt(-1)

## Conditional square root

To be able to control which square roots in the complete expression should be evaluatable for negative values, one could use {class}`~sympy.functions.elementary.piecewise.Piecewise`:

In [ ]:
def complex_sqrt(x: sp.Symbol) -> sp.Expr:
    return sp.Piecewise(
        (sp.sqrt(-x) * sp.I, x < 0),
        (sp.sqrt(x), True),
    )


complex_sqrt(x)

In [ ]:
display(
    complex_sqrt(-4),
    complex_sqrt(+4),
)

Be careful though when lambdifying this expression: _do not use the `__dict__` of the {mod}`numpy` module as backend_, but use the module itself instead. When using `__dict__`, {func}`~sympy.utilities.lambdify.lambdify` will return an `if-else` statement, which is inefficient and, worse, will result in problems with {doc}`JAX <jax:index>`:

:::{warning}

Do not use the module `__dict__` for the `modules` argument of {func}`~sympy.utilities.lambdify.lambdify`.

:::

In [ ]:
np_complex_sqrt_no_select = sp.lambdify(x, complex_sqrt(x), np.__dict__)
source = inspect.getsource(np_complex_sqrt_no_select)
print(source)

In [ ]:
np_complex_sqrt_no_select(-1)

In [ ]:
jax_complex_sqrt_no_select = jax.jit(np_complex_sqrt_no_select)
jax_complex_sqrt_no_select(-1)

When instead using the {mod}`numpy` module (or `"numpy"`), {func}`~sympy.utilities.lambdify.lambdify` correctly lambdifies to {func}`numpy.select` to represent the cases.

In [ ]:
np_complex_sqrt = sp.lambdify(x, complex_sqrt(x), np)
source = inspect.getsource(np_complex_sqrt)

In [ ]:
print(format_str(source.replace("nan)", "nan,)"), mode=FileMode()))

Still, JAX does not handle this correctly. First, lambdifying JAX again results in this `if-else` syntax:

In [ ]:
jnp_complex_sqrt = sp.lambdify(x, complex_sqrt(x), jnp)
source = inspect.getsource(jnp_complex_sqrt)
print(source)

But even if we lambdify to {mod}`numpy` and decorate the result with a {func}`jax.jit` decorator, the resulting function does not work properly:

In [ ]:
jax_complex_sqrt_error = jax.jit(np_complex_sqrt)
source = inspect.getsource(jax_complex_sqrt_error)

In [ ]:
print(format_str(source.replace("nan)", "nan,)"), mode=FileMode()))

In [ ]:
jax_complex_sqrt_error(-1)

The very same function in created purely with {mod}`jax.numpy` does work without problems, so it seems this is a SymPy problem:

In [ ]:
@jax.jit
def jax_complex_sqrt(x):
    return jnp.select(
        [jnp.less(x, 0), True],
        [1j * jnp.sqrt(-x), jnp.sqrt(x)],
        default=jnp.nan,
    )

In [ ]:
jax_complex_sqrt(sample)